In [8]:
import pandas as pd

df = pd.read_excel("../../data/df_de_final_duplikate_markiert.xlsx", engine="openpyxl")


In [9]:
# Neue Spalte für Duplikat-Clustering
df["Cluster_Duplikate"] = -1

# Cluster-Nummern zuweisen für jede Duplikat_Gruppe
for idx, gruppe in enumerate(df["Duplikat_Gruppe"].dropna().unique()):
    df.loc[df["Duplikat_Gruppe"] == gruppe, "Cluster_Duplikate"] = idx


In [10]:
# Nur eine Frage pro Duplikat_Gruppe behalten (oder Singletons)
df_unique = df.drop_duplicates(subset=["Duplikat_Gruppe"], keep="first").copy()

print(f"Anzahl einzigartiger Fragen: {len(df_unique)}")


Anzahl einzigartiger Fragen: 1316


In [11]:
import spacy
import numpy as np

# spaCy Modell laden
nlp = spacy.load('de_core_news_lg')

# Normierte Fragetexte verwenden
texts = df_unique['Frage_Text'].str.lower().str.strip().tolist()
docs = list(nlp.pipe(texts))
vectors = np.array([doc.vector for doc in docs])


In [12]:
from sklearn.metrics.pairwise import cosine_distances

distance_matrix = cosine_distances(vectors).astype(np.float64)


In [13]:
import hdbscan

clusterer = hdbscan.HDBSCAN(min_cluster_size=2, metric="precomputed")
labels_hdbscan = clusterer.fit_predict(distance_matrix)

# Cluster-Labels in DataFrame speichern
df_unique["Cluster_HDBSCAN"] = labels_hdbscan


In [14]:
# Mapping: ID_gesamt --> Cluster_HDBSCAN
id_to_cluster = df_unique.set_index("ID_gesamt")["Cluster_HDBSCAN"].to_dict()

def assign_final_cluster(row):
    if row["Cluster_Duplikate"] != -1:
        return row["Cluster_Duplikate"]
    else:
        return id_to_cluster.get(row["ID_gesamt"], -1)

df["Cluster_Final"] = df.apply(assign_final_cluster, axis=1)


In [15]:
df.to_excel("../../data/df_de_final_clustered.xlsx", index=False, engine="openpyxl")


In [16]:
import pandas as pd

# Ergebnis-DataFrame laden
df = pd.read_excel("../../data/df_de_final_clustered.xlsx", engine="openpyxl")

# 1. Alle Fragen, die KEIN Duplikat waren (Cluster_Duplikate == -1)
einzigartige_fragen = df[df["Cluster_Duplikate"] == -1]

# 2. Innerhalb der einzigartigen Fragen:
# Welche sind durch HDBSCAN in ein echtes Cluster (> -1) gekommen?
geclusterte_einzigartige_fragen = einzigartige_fragen[einzigartige_fragen["Cluster_Final"] != -1]

# 3. Anzahl berechnen
anzahl_einzigartige = len(einzigartige_fragen)
anzahl_geclusterte_einzigartige = len(geclusterte_einzigartige_fragen)

# 4. Prozentsatz
effektivitätsquote = anzahl_geclusterte_einzigartige / anzahl_einzigartige * 100

# Ausgabe
print(f"Anzahl einzigartiger Fragen insgesamt: {anzahl_einzigartige}")
print(f"Anzahl einzigartiger Fragen, die durch HDBSCAN geclustert wurden: {anzahl_geclusterte_einzigartige}")
print(f"Effektivität von HDBSCAN (in %): {effektivitätsquote:.2f}%")


Anzahl einzigartiger Fragen insgesamt: 1901
Anzahl einzigartiger Fragen, die durch HDBSCAN geclustert wurden: 1
Effektivität von HDBSCAN (in %): 0.05%
